# Lecture 32: Iterative Local Search

---

## Overview

As the name suggests, the Iterative Local Search algorithm repeatedly deploys local search heuristics until convergence. In each such iteration, the algorithm transforms the current solution – s, to generate a new solution – s', in its neighbourhood – N. The algorithm then performs a local search on this new solution using one of the local search heuristics – $\text{LS}$, such as the Hill Climb, Tabu Search, Threshold Accepting, Simulated Annealing, etc, given search parameters – χ. The Iterative Local Search algorithm then updates the current and best solution based on quality of the resulting new solution, determined by the objective function value – f. Finally, upon convergence, determined by a threshold level of solution quality or computation effort, the algorithm returns the best solution.

The Iterative Local Search algorithm effectively balances exploration and exploitation. In particular, it ensures solution landscape exploration (diversification) through the neighbourhood transformations that render new random solutions to escape local optima traps. On the other hand, the algorithm ensures solution landscape exploitation (intensification) through the local search heuristics that refine solutions to reach local optima. Hence, the algorithm necessitates deliberate development contextual to the problem at hand with suitable neighbourhood structure, appropriate local search procedure, and sophisticated parameter tuning. Thus, the literature has sparsely deployed Iterative Local Search algorithm for optimisation in the context of ITS; some examples include vehicle navigation, vehicle routing, and traffic signal optimisation.

---

## Pseudo Code

1. **Procedure** $\text{ILS}(s_o, (\text{LS}, N, \chi))$
2. $s ← s_o$ &emsp;<small>// initialise current solution $s$ as the initial solution $s_o$</small>
3. $s^* ← s$ &emsp;<small>// initialise best solution $s^*$ as the current solution</small>
4. **while** $!\text{converged}$ **do** &emsp;<small>// repeat until converged</small>
5. &emsp;$s' {R \atop ←} N(s)$ &emsp;<small>// generate new random solution from neighborhood $N$ of the current solution</small>
6. &emsp;$s' ← \text{LS}(s', (N, \chi))$ &emsp;<small>// perform local search $\text{LS}$ on the new solution given search parameters $\chi$</small>
7. &emsp;**if** $f(s') < f(s)$ **then** &emsp;<small>// if the new solution is better than the current solution</small>
8. &emsp;&emsp;$s ← s'$ &emsp;<small>// update the current solution to the new solution</small>
9. &emsp;**end if**
10. &emsp;**if** $f(s) < f(s^*)$ **then** &emsp;<small>// if the current solution is better than the best solution</small>
11. &emsp;&emsp;$s^* ← s$ &emsp;<small>// update the best solution to the current solution</small>
12. &emsp;**end if**
13. **end while**
14. **return** $s^*$ &emsp;<small>// return the best solution</small>

---

## Implementation

---

## Case Study

---

```{note}
...
```